<a href="https://colab.research.google.com/github/AtaOzkan/n/blob/main/covid_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns

from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, InputLayer, RandomRotation, RandomFlip, RandomContrast, Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, CategoricalAccuracy

# ***DATASET DOWNLOADING***

In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pranavraikokte/covid19-image-dataset

Dataset URL: https://www.kaggle.com/datasets/pranavraikokte/covid19-image-dataset
License(s): CC-BY-SA-4.0
 87% 137M/158M [00:01<00:00, 110MB/s]
100% 158M/158M [00:01<00:00, 115MB/s]


In [ ]:
!unzip "/content/covid19-image-dataset.zip" -d "/content/covid19-image-dataset"

Archive:  /content/covid19-image-dataset.zip
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0100.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0102.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0105.png  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0106.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0108.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0111.jpg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0112.jpg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0113.jpg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0115.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0118.jpeg  
  inflating: /content/covid19-image-dataset/Covid19-dataset/test/Covid/0119.jpeg  
  inflating: /content/covid19-image-dataset/Co

# ***DATA PREPROCESSİNG***

In [ ]:
train_directory="/content/covid19-image-dataset/Covid19-dataset/train"
validation_directory="/content/covid19-image-dataset/Covid19-dataset/test"
CLASS_NAMES=["Covid","Normal","Viral Pneumonia"]

CONFIGURATION={
    "IM_SIZE":224,
    "BATCH_SIZE":32,
    "N_DENSE1":1024,
    "N_DENSE2":128,
    "N_EPOCH":10,
    "N_CLASSES":3,
}

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=87,
)

Found 251 files belonging to 3 classes.


In [ ]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=87,
)

Found 66 files belonging to 3 classes.


# **VISUALIZATION**

In [ ]:
plt.figure(figsize=(10,10))
for images,labels in train_dataset.take(1):
  for i in range(16):
    ax= plt.subplot(4,4,i+1)
    plt.imshow(images[i]/255.)
    plt.title(CLASS_NAMES[tf.argmax(labels[i], axis = 0).numpy()])
    plt.axis("off")

# ***MODEL***

In [ ]:
backbone = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"],3),
    pooling=None,
    classes=3,
    classifier_activation='softmax',
)

In [ ]:
backbone.trainable = False

In [ ]:
model= tf.keras.Sequential([
    InputLayer(input_shape=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"],3)),
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(CONFIGURATION["N_DENSE1"], activation='relu'),
    BatchNormalization(),
    tf.keras.layers.Dense(CONFIGURATION["N_DENSE2"], activation='relu'),
    tf.keras.layers.Dense(CONFIGURATION["N_CLASSES"], activation='softmax')
])
model.summary()

# ***FINETUNING***

In [ ]:
backbone.trainable = True

In [ ]:
input = Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"],3))

x = backbone(input, training = False)
x = GlobalAveragePooling2D()(x)
x = Dense( CONFIGURATION["N_DENSE1"], activation = "relu")(x)
x = BatchNormalization()(x)
x = Dense( CONFIGURATION["N_DENSE2"], activation = "relu")(x)
output = Dense( CONFIGURATION["N_CLASSES"], activation = "softmax")(x)

f_model = Model(input, output)
f_model.summary()

# ***TRAİNİNG***

In [ ]:
callbacks= ModelCheckpoint(
    "best_weights",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,

)

In [ ]:
loss_function= CategoricalCrossentropy()
metrics = [CategoricalAccuracy(name = "accuracy")]

In [ ]:
f_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001/100),
    loss=loss_function,
    metrics=metrics
)

In [ ]:
history= f_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=CONFIGURATION["N_EPOCH"],
    verbose=1,
    callbacks=[callbacks]
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train_loss','validation_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('acc')
plt.xlabel('epochs')
plt.legend(['train_acc','validation_acc'])
plt.show()

In [ ]:
f_model.load_weights('best_weights')

In [ ]:
f_model.evaluate(validation_dataset)

# ***TESTİNG***

In [ ]:
plt.figure(figsize = (12,12))

for images, labels in validation_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4,4, i+1)
    plt.imshow(images[i]/255.)

    plt.title("True Label - : " + CLASS_NAMES[tf.argmax(labels[i], axis = -1).numpy()]
              + "\n" + "Predicted Label - : "
              + CLASS_NAMES[int(tf.argmax(model(tf.expand_dims(images[i], axis = 0)), axis =-1).numpy()[0])] )
    plt.axis("off")

# **CM**

In [ ]:
labels=[]
predicted=[]

for im,label in validation_dataset:
  predicted.append(model(im))
  labels.append(label.numpy())

In [ ]:
pred = np.concatenate([np.argmax(predicted[:-1], axis = -1).flatten(), np.argmax(predicted[-1], axis = -1).flatten()])
lab = np.concatenate([np.argmax(labels[:-1], axis = -1).flatten(), np.argmax(labels[-1], axis = -1).flatten()])

In [ ]:
cm= confusion_matrix(lab,pred)
print(cm)
plt.figure(figsize=(8,8))

sns.heatmap(cm,annot=True)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()


# ***DATA AUGMENTATION***

In [ ]:
augment_layers= tf.keras.Sequential([
    RandomFlip(mode='horizontal'),
    RandomRotation(factor=(-0.025, 0.025)),
    RandomContrast(factor=(0,1))
])

In [ ]:
def augment_layer(image,label):
  return augment_layers(image,training=True),label

In [ ]:
train_dataset=(
    train_dataset
    .map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
validation_dataset=(
    validation_dataset.prefetch(tf.data.AUTOTUNE)
)